In [1]:
import pandas as pd
import os
import numpy as np
import keras as K
import pydicom
import cv2
from pathlib import Path
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from PIL import Image

Using TensorFlow backend.


**READ IN PATIENT INFO**

In [2]:
#Read 
patient_info = pd.read_csv(Path('Methylation/patient_info.csv').absolute())

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,9,10,11,23,25,27,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
patient_info = patient_info[['Patient ID','modality_path','GivenModalityForThisStudy','ModalityName']].groupby(['Patient ID','modality_path','GivenModalityForThisStudy']).count().reset_index()
patient_info = patient_info.rename(columns={'ModalityName': 'count'})
patient_info = patient_info.drop_duplicates('modality_path')

In [4]:
len(patient_info.loc[patient_info['GivenModalityForThisStudy']=='t1'])

131

In [5]:
from custom_metrics import dice_coef, soft_dice_coef, dice_coef_loss, combined_dice_ce_loss

In [6]:
custom_objects = {
"combined_dice_ce_loss": combined_dice_ce_loss,
"dice_coef_loss": dice_coef_loss,
"dice_coef": dice_coef,
"soft_dice_coef": soft_dice_coef}

**LOAD TRAINED BRATS UNET MODEL**

In [7]:
import keras as K
unet_model = K.models.load_model(str(Path('unet/2D/output/unet_model_for_decathlon.hdf5').absolute()), custom_objects=custom_objects)

In [8]:
input_channels = {
     "flair": 0,
     "t1": 1,
     "t1Gd": 2,
     "t2": 3
 }

In [9]:
#Subset T1
patient_info_t1 = patient_info.loc[patient_info['GivenModalityForThisStudy']=='t1']

In [10]:
patient_info.head()

,Patient ID,modality_path,GivenModalityForThisStudy,count
0,TCGA-02-0003,/external_drive/MRI_methylation/Methylation/da...,t2,25
1,TCGA-02-0003,/external_drive/MRI_methylation/Methylation/da...,flair,25
2,TCGA-02-0003,/external_drive/MRI_methylation/Methylation/da...,t1,25
3,TCGA-02-0003,/external_drive/MRI_methylation/Methylation/da...,t1Gd,25
4,TCGA-02-0006,/external_drive/MRI_methylation/Methylation/da...,t2,23


**USE TRAINED UNET TO PREDICT ON TCGA DATA**

In [11]:
frames_with_tumour = []
for idx, row in patient_info_t1.iterrows():
    MRI_vid = []
    patient_frames_w_tumour = []
    for dicom in os.listdir(row['modality_path']):
        dcm = pydicom.dcmread(os.path.join(row['modality_path'],dicom))
        img = dcm.pixel_array
        try:
            img = dcm.pixel_array*dcm.RescaleSlope+dcm.RescaleIntercept
        except:
            pass
        try:
            min_value, max_value = dcm.WindowCenter - dcm.WindowWidth/2, dcm.WindowCenter + dcm.WindowWidth/2
            img[img < min_value] = min_value
            img[img > max_value] = max_value
        except:
            pass
#         img = cv2.resize(img, (144,144))
        if (np.mean(img)<20) & (np.std(img)<20):
            pass
        else:
            original_img = img.astype(np.float32)
            img = (img - np.mean(img))/np.std(img)
            img = img.reshape(img.shape+(1,))
            pred = unet_model.predict(np.array([img]))[0,:,:,0]
            tumour_size = len(np.where(pred.flatten()>0.5)[0])
#             if tumour_size>(0.01*dcm.pixel_array.shape[0]*dcm.pixel_array.shape[1]):
            if tumour_size>0:
                patient_frames_w_tumour.append(dicom)
            #Save images to disk
            if tumour_size>0:
                mask = np.where(pred>0.5,np.max(original_img.flatten()),0.0).astype(np.float32)
                plt.figure(figsize=(10, 6.67))
                plt.subplot(1, 2, 1)
                plt.imshow(original_img, cmap="bone", origin="lower")
                plt.title("MRI")
                plt.axis("off")
                plt.subplot(1, 2, 2)
                plt.imshow(mask, origin="lower")
                plt.title("Prediction")
                plt.axis("off")
                os.makedirs(row['modality_path'].replace('TCGA-GBM','TCGA-GBM-tumour-mask'),exist_ok=True)
                plt.savefig(os.path.join(row['modality_path'].replace('TCGA-GBM','TCGA-GBM-tumour-mask'),
                                     dicom.replace('dcm','jpg')), bbox_inches='tight')                    
#                 overlayed_mask = cv2.addWeighted(original_img,1.0,mask,0.5,0.0).astype(np.uint8)
#                 im = Image.fromarray(overlayed_mask)
#                 im.save(os.path.join(row['modality_path'].replace('TCGA-GBM','TCGA-GBM-tumour-mask'),
#                                      dicom.replace('dcm','jpg')))             
    frames_with_tumour.append(sorted(patient_frames_w_tumour))

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/pydicom/filereader.py:313: UserWarning: Expected explicit VR, but found implicit VR - using implicit VR for reading
  warnings.warn(message, UserWarning)


In [12]:
patient_info_t1['frames_with_tumour'] = frames_with_tumour

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [13]:
patient_info_t1.head()

,Patient ID,modality_path,GivenModalityForThisStudy,count,frames_with_tumour
2,TCGA-02-0003,/external_drive/MRI_methylation/Methylation/da...,t1,25,"[000000.dcm, 000001.dcm, 000002.dcm, 000003.dc..."
6,TCGA-02-0006,/external_drive/MRI_methylation/Methylation/da...,t1,23,"[000000.dcm, 000001.dcm, 000002.dcm, 000003.dc..."
10,TCGA-02-0009,/external_drive/MRI_methylation/Methylation/da...,t1,23,"[000000.dcm, 000001.dcm, 000002.dcm, 000003.dc..."
14,TCGA-02-0011,/external_drive/MRI_methylation/Methylation/da...,t1,23,"[000000.dcm, 000001.dcm, 000002.dcm, 000003.dc..."
18,TCGA-02-0033,/external_drive/MRI_methylation/Methylation/da...,t1,23,"[000000.dcm, 000001.dcm, 000002.dcm, 000003.dc..."


**SUBSET PATIENT INFO TO INCLUDE ONLY T1 PATIENTS AND SAVE TO DISK**

In [14]:
patient_info_t1.to_pickle(str(Path('Methylation/patient_info_t1.pickle').absolute()))